In [5]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [6]:
#Getting Chrome Driver and browser
executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless = False)

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
#using BeautifulSoup to write it into html
html = browser.html
soup = BeautifulSoup(html,"html.parser")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17134 x86_64)


In [ ]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

In [ ]:
# Getting the most recent news title and paragraph
latest_news_title = soup.find("div",class_="content_title").text
latest_news_paragraph = soup.find("div", class_="article_teaser_body").text
print(f"Title: {latest_news_title}")
print(f"Para: {latest_news_paragraph}")

#JPL Image

In [ ]:
# Go to the JPL website
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_image)

In [ ]:
#Use BeautifulSoup to write it into html
html_image = browser.html
image_soup = BeautifulSoup(html_image,"html.parser")#lxml
print(soup.prettify())

In [ ]:
# Find image url for current featured mars image
featured_image = (image_soup.find('div', class_='default floating_text_area ms-layer')).find('footer')
featured_image_url = 'https://www.jpl.nasa.gov'+ featured_image.find('a')['data-fancybox-href']
print(str(featured_image_url))

#Mars Weather

In [ ]:
# Go to the Mars Weather website
url_mars_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_mars_weather)

In [ ]:
html_mars_weather = browser.html
mars_weather_soup = BeautifulSoup(html_mars_weather,"html.parser")#lxml
print(mars_weather_soup.prettify())


In [ ]:
#Get most recent tweet.
mars_weather = mars_weather_soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)

#Mars Facts

In [ ]:
url_facts = "http://space-facts.com/mars/"
mars_facts_df = pd.read_html(url_facts)[0]
mars_facts_df.columns=['Attribute','Value']
mars_facts_df.set_index(['Attribute'])

In [ ]:
mars_facts_html_table = mars_facts_df.to_html()
mars_facts_html_table = mars_facts_html_table.replace("\n", "")
mars_facts_html_table

#Mars Hemisphere

In [7]:
# Go to the Mars Hemisphere website
url_mars_hemi = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_mars_hemi)

mars_hemi_html = browser.html
mars_hemi_soup = BeautifulSoup(mars_hemi_html, "html.parser")#lxml
print(mars_hemi_soup.prettify())

<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <link href="//ajax.googleapis.com/ajax/libs/jqueryui/1.11.3/themes/smoothness/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <script async="" src="https://ssl.google-analytics.com/ga.js" type="text/javascript">
  </script>
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js" type="text/javascript">
  </script>
  <title>
   Astropedia Search Results | USGS Astrogeology Science Center
  </title>
  <meta content="USGS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="USGS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="x61hXXVj7wtfBSNOPnTftajMsZ5yB2W-qRo

In [8]:
#Get image items on page. These coordinate with the 4 hemispheres
base_url ="https://astrogeology.usgs.gov"

images_array = mars_hemi_soup.find_all('div', class_='item')

# Create list to store dictionaries of data
hemi_image_urls = []
images_array

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiapa

In [9]:
# Loop through each hemisphere and click on link to find large resolution image url
for image in images_array:
    href = image.find('a', class_='itemLink product-item')
    link = base_url + href['href']
    browser.visit(link)

    time.sleep(1)

    hemi_html2 = browser.html
    hemi_soup2 = BeautifulSoup(hemi_html2, "html.parser")

    hemi_title = hemi_soup2.find('div', class_='content').find('h2', class_='title').text
    if hemi_title.endswith('Enhanced'):
        hemi_title = hemi_title[:-9]
    
    hemisphere_info = {'title': hemi_title,\
                       'img_url': hemi_soup2.find('div', class_='downloads').find('a')['href']}
    
    # Append dictionary to list
    hemi_image_urls.append(hemisphere_info)

hemi_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]